In [184]:
import requests
import re
import pandas as pd
import numpy as np

In [185]:
with open('api.txt') as file:
    api_key = file.read().strip()
query = "\"Fat and Oils\""


fats = []
for page in range(1,6):
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?api_key={api_key}&query={query}&pageNumber={page}'
    response = requests.get(url)
    fat_rows = response.json()
    fats.extend(fat_rows)

fatsdf = pd.DataFrame(fats)

In [186]:
fatsdf = fatsdf.sort_values('description')
fatsdf = fatsdf[fatsdf['dataType']=='SR Legacy']
fatsdf = fatsdf.drop(columns=[ 'publicationDate', 'brandOwner', 'gtinUpc'])
fatsdf.reset_index()

,index,fdcId,description,dataType,ndbNumber,foodNutrients
0,2,172345,"Animal fat, bacon grease",SR Legacy,4609,"[{'number': '319', 'name': 'Retinol', 'amount'..."
1,3,169894,"Butter replacement, without fat, powder",SR Legacy,43214,"[{'number': '324', 'name': 'Vitamin D (D2 + D3..."
2,4,173581,"Butter, light, stick, with salt",SR Legacy,4601,"[{'number': '262', 'name': 'Caffeine', 'amount..."
3,5,172344,"Butter, light, stick, without salt",SR Legacy,4602,"[{'number': '334', 'name': 'Cryptoxanthin, bet..."
4,6,167684,"Creamy dressing, made with sour cream and/or b...",SR Legacy,42116,"[{'number': '430', 'name': 'Vitamin K (phylloq..."
...,...,...,...,...,...,...
211,221,173575,"Shortening, special purpose for baking, soybea...",SR Legacy,4587,"[{'number': '268', 'name': 'Energy', 'amount':..."
212,222,173574,"Shortening, special purpose for cakes and fros...",SR Legacy,4586,"[{'number': '268', 'name': 'Energy', 'amount':..."
213,223,173584,"Shortening, vegetable, household, composite",SR Legacy,4615,"[{'number': '324', 'name': 'Vitamin D (D2 + D3..."
214,224,171422,"Vegetable oil, palm kernel",SR Legacy,4513,"[{'number': '431', 'name': 'Folic acid', 'amou..."


In [ ]:
def extract_nutrient(nutrient_list, target):
    if nutrient_list is None:
        return None
    if len(nutrient_list) == 0:
        return None
    for nutrient in nutrient_list:
        if nutrient.get("name") == target:
            return nutrient.get("amount")
    return None


fatsdf["energy(Kcal)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Energy"))
fatsdf["protein(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Protein"))
fatsdf["saturated_fat(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Fatty acids, total saturated"))
fatsdf["trans_fat(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Fatty acids, total trans"))
fatsdf["monounsat_fat(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Fatty acids, total monounsaturated"))
fatsdf["polyunsat_fat(g)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Fatty acids, total polyunsaturated"))
fatsdf["cholesterol(mg)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Cholesterol"))
fatsdf["sodium(mg)"] = fatsdf["foodNutrients"].apply(lambda items: extract_nutrient(items, "Sodium, Na"))

In [ ]:
fatsdf = fatsdf.fillna(0.0)
fatsdffinal = fatsdf.rename(columns={'description': 'Fat/Oil Name'})
fatsdffinal = fatsdffinal.drop(columns=['fdcId', 'foodNutrients', 'dataType', 'ndbNumber'])

216

In [210]:
fatsdffinal.to_csv('fatandoils.csv')

links: 
- https://fdc.nal.usda.gov/api-guide
- https://fdc.nal.usda.gov/food-search?type=SR%20Legacy&SRFoodCategory=Fats%20and%20Oils
- https://app.swaggerhub.com/apis/fdcnal/food-data_central_api/1.0.1#/FDC/getFoodsList